In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')


In [ ]:
data=pd.read_csv("indian_liver_patient.csv")
data.head()

,Age,Gender,Total_Bilirubin,Direct_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Total_Protiens,Albumin,Albumin_and_Globulin_Ratio,Dataset
0,65,Female,0.7,0.1,187,16,18,6.8,3.3,0.90,1
1,62,Male,10.9,5.5,699,64,100,7.5,3.2,0.74,1
2,62,Male,7.3,4.1,490,60,68,7.0,3.3,0.89,1
3,58,Male,1.0,0.4,182,14,20,6.8,3.4,1.00,1
4,72,Male,3.9,2.0,195,27,59,7.3,2.4,0.40,1


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 583 entries, 0 to 582
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Age                         583 non-null    int64  
 1   Gender                      583 non-null    object 
 2   Total_Bilirubin             583 non-null    float64
 3   Direct_Bilirubin            583 non-null    float64
 4   Alkaline_Phosphotase        583 non-null    int64  
 5   Alamine_Aminotransferase    583 non-null    int64  
 6   Aspartate_Aminotransferase  583 non-null    int64  
 7   Total_Protiens              583 non-null    float64
 8   Albumin                     583 non-null    float64
 9   Albumin_and_Globulin_Ratio  579 non-null    float64
 10  Dataset                     583 non-null    int64  
dtypes: float64(5), int64(5), object(1)
memory usage: 50.2+ KB


In [ ]:
data["Gender"]=data["Gender"].replace({"Male":0,"Female":0})

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 583 entries, 0 to 582
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Age                         583 non-null    int64  
 1   Gender                      583 non-null    int64  
 2   Total_Bilirubin             583 non-null    float64
 3   Direct_Bilirubin            583 non-null    float64
 4   Alkaline_Phosphotase        583 non-null    int64  
 5   Alamine_Aminotransferase    583 non-null    int64  
 6   Aspartate_Aminotransferase  583 non-null    int64  
 7   Total_Protiens              583 non-null    float64
 8   Albumin                     583 non-null    float64
 9   Albumin_and_Globulin_Ratio  579 non-null    float64
 10  Dataset                     583 non-null    int64  
dtypes: float64(5), int64(6)
memory usage: 50.2 KB


In [ ]:
from imblearn.over_sampling import RandomOverSampler
X=data.iloc[:,:-1]
y=data.iloc[:,-1]
# Apply random oversampling to balance the classes
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X, y)

# Check the number of instances in the resampled data
print("Resampled dataset size:", X_resampled.shape, y_resampled.shape)
print("Number of positive cases after resampling:", sum(y_resampled == 1))
print("Number of negative cases after resampling:", sum(y_resampled == 2))

Resampled dataset size: (832, 10) (832,)
Number of positive cases after resampling: 416
Number of negative cases after resampling: 416


In [ ]:
import numpy as np

# Function to replace outliers with the median
def replace_outliers_with_median(df):
    df = df.copy()
    for column in df.columns:
        if df[column].dtype in [np.float64, np.int64]:
            mean = df[column].mean()
            std = df[column].std()
            z_scores = (df[column] - mean) / std
            outliers = np.abs(z_scores) > 3
            median = df[column].median()
            df.loc[outliers, column] = median
    return df

# Function to replace missing values with the median
def replace_missing_with_median(df):
    df = df.copy()
    for column in df.columns:
        if df[column].dtype in [np.float64, np.int64]:
            median = df[column].median()
            df[column].fillna(median, inplace=True)
    return df


In [ ]:
# Replace outliers in the resampled features
X_resampled = replace_outliers_with_median(X_resampled)

# Replace missing values in the resampled features
X_resampled = replace_missing_with_median(X_resampled)


In [ ]:
#
from sklearn.preprocessing import StandardScaler

# Standardize the data
scaler = StandardScaler()
X_resampled_scaled = scaler.fit_transform(X_resampled)


In [ ]:
from sklearn.decomposition import PCA

# Apply PCA
pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X_resampled_scaled)


In [ ]:
from sklearn.decomposition import FactorAnalysis

# Apply Factor Analysis
fa = FactorAnalysis(n_components=X_pca.shape[1])
X_fa = fa.fit_transform(X_resampled_scaled)


In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Apply LDA
lda = LinearDiscriminantAnalysis(n_components=1)
X_lda = lda.fit_transform(X_resampled_scaled, y_resampled)


In [ ]:
import numpy as np

# Combine PCA, FA, and LDA features
S = np.hstack((X_pca, X_fa, X_lda))

# Check the shape of the combined feature matrix
print("Combined feature matrix shape:", S.shape)


Combined feature matrix shape: (832, 15)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt


In [ ]:
# Define the models
models = {
    "Logistic Regression": LogisticRegression(),
    "Random Forest": RandomForestClassifier(random_state=42),
    "SVM": SVC(),
    "MLP": MLPClassifier(random_state=42),
    "knn": KNeighborsClassifier()
}



In [ ]:
from sklearn.model_selection import cross_val_score

# Apply 10-fold cross-validation and evaluate the models
results = {}
for model_name, model in models.items():
    scores = cross_val_score(model, S, y_resampled, cv=10, scoring='accuracy')
    results[model_name] = scores
    print(f"{model_name} Accuracy: {scores.mean():.4f} (+/- {scores.std():.4f})")


Logistic Regression Accuracy: 0.7019 (+/- 0.0793)
Random Forest Accuracy: 0.8558 (+/- 0.0454)
SVM Accuracy: 0.7067 (+/- 0.0723)
MLP Accuracy: 0.7247 (+/- 0.0489)
knn Accuracy: 0.7127 (+/- 0.0608)


In [ ]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(S,y_resampled,test_size=0.25,random_state=42,stratify=y_resampled)
knn = KNeighborsClassifier()
mlp = MLPClassifier(random_state=42)
rf = RandomForestClassifier(random_state=42)
lr = LogisticRegression()
svm = SVC()

# Ensemble method (Voting Classifier)
ensemble = VotingClassifier(estimators=[
    ('knn', knn),
    ('mlp', mlp),
    ('rf', rf),
    ('lr', lr),
    ('svm', svm)], voting='hard')


In [ ]:
# Train and evaluate each model
for model_name, model in models.items():
    model.fit(xtrain, ytrain)  # Train the model
    ypred = model.predict(xtest)  # Predict on the test set

    # Check if model has predict_proba method; otherwise, use decision_function
    if hasattr(model, "predict_proba"):
        yproba = model.predict_proba(xtest)[:, 1]  # Probability estimates for the positive class
    else:  # For models like SVM which might not have predict_proba
        if hasattr(model, "decision_function"):
            yproba = model.decision_function(xtest)
        else:
            yproba = None  # Set to None if neither method is available

    # Calculate AUC only if yproba is available
    if yproba is not None:
        auc = roc_auc_score(ytest, yproba)  # Calculate AUC
    else:
        auc = "N/A"  # Not applicable if probabilities are not available

    accuracy = accuracy_score(ytest, ypred)  # Calculate accuracy
    print(f"{model_name} Accuracy: {accuracy:.4f}")
    print(f"{model_name} AUC: {auc}")
    #print(f"{model_name} Classification Report:\n {classification_report(ytest, ypred)}\n")


KNN Accuracy: 0.7404
KNN AUC: 0.7785687869822484
MLP Accuracy: 0.7067
MLP AUC: 0.7140347633136095
Random Forest Accuracy: 0.7885
Random Forest AUC: 0.9018121301775148
Logistic Regression Accuracy: 0.6779
Logistic Regression AUC: 0.7054363905325445
SVM Accuracy: 0.6779
SVM AUC: 0.6996116863905325
Ensemble Accuracy: 0.7212
Ensemble AUC: N/A
